In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time


In [ ]:
def get_matches(date: str):
    """Scrape matches from yallakora for a given date and return as list of dicts."""
    url = f"https://www.yallakora.com/match-center/?date={date}"
    response = requests.get(url)
    response.raise_for_status()  #   Stops the code if there's a connection error

    soup = BeautifulSoup(response.content, "lxml")
    championships = soup.find_all("div", {"class": "matchCard"})

    if not championships:
        print(f"⚠️ مفيش مباريات في {date}")
        return []

    matches_details = []

    for championship in championships:
        try:
            championship_title = championship.find("h2").text.strip()
        except AttributeError:
            championship_title = "غير معروف"

        all_matches = championship.find_all("div", {"class": "item"})

        for match in all_matches:
            try:
                team_a = match.find("div", {"class": "teamA"}).text.strip()
                team_b = match.find("div", {"class": "teamB"}).text.strip()

                scores = match.find("div", {"class": "MResult"}).find_all(
                    "span", {"class": "score"})
                if len(scores) == 2:
                    score = f"{scores[0].text.strip()} - {scores[1].text.strip()}"
                else:
                    score = "لم تبدأ"

                time = match.find("span", {"class": "time"}).text.strip(
                ) if match.find("span", {"class": "time"}) else ""

                # رابط المباراة
                match_link = match.find("a")["href"] if match.find("a") else ""
                if match_link and not match_link.startswith("http"):
                    match_link = "https://www.yallakora.com" + match_link

                matches_details.append({
                    "البطولة": championship_title,
                    "الفريق الأول": team_a,
                    "الفريق الثاني": team_b,
                    "النتيجة": score,
                    "الوقت": time,
                    "رابط المباراة": match_link
                })
            except Exception as e:
                print(f"⚠️ حصل خطأ في استخراج ماتش: {e}")
                continue

    return matches_details

In [3]:
def save_to_csv(matches, date: str):
    """Save matches list to CSV using pandas."""
    if not matches:
        print("⚠️ مفيش داتا للحفظ")
        return

    df = pd.DataFrame(matches)
    path = f"D:/python_work/web_scraping_projects/yalakora/matches_{date}.csv"
    df.to_csv(path, index=False, encoding="utf-8-sig")
    print(f"✅ تم حفظ {len(matches)} ماتش في الملف: {path}")

In [6]:
def main():
    date = input("من فضلك أدخل التاريخ بصيغة YYYY-MM-DD: ").strip()
    matches = get_matches(date)
    save_to_csv(matches, date)


if __name__ == "__main__":
    main()

✅ تم حفظ 45 ماتش في الملف: D:/python_work/web_scraping_projects/yalakora/matches_2025-9-28.csv
